In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import xgboost as xgb
import lightgbm as lgb
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df_train = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/train.csv")

data1 = pd.read_csv("/kaggle/input/hackbogazicicop/data4.csv")
data2 = pd.read_csv("/kaggle/input/hackbogazicicop/data5.csv")
data3 = pd.read_csv("/kaggle/input/hackbogazicicop/data6.csv")

data1["is_faulty"] = data1["is_faulty"].astype(int)
data2["is_faulty"] = data2["is_faulty"].astype(int)
data3["is_faulty"] = data3["is_faulty"].astype(int)

In [ ]:
data1.head()

In [ ]:
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(np.abs(corrmat), vmax=.8, square=True);

In [ ]:
df_train['date_time'] = pd.to_datetime(df_train['date_time'])
df_train['year'] = df_train['date_time'].dt.year
df_train['month'] = df_train['date_time'].dt.month
df_train['week'] = df_train['date_time'].dt.week
df_train['day'] = df_train['date_time'].dt.day
df_train['dayofweek'] = df_train['date_time'].dt.dayofweek
df_train['time'] = df_train['date_time'].dt.date - df_train['date_time'].dt.date.min()
df_train['hour'] = df_train['date_time'].dt.hour
df_train['time'] = df_train['time'].apply(lambda x : x.days)
df_train.drop(columns = 'date_time', inplace = True)
df_train['SMC'] = (df_train['absolute_humidity'] * 100) / df_train['relative_humidity']
df_train['Dew_Point'] = 243.12*(np.log(df_train['relative_humidity'] * 0.01) + (17.62 * df_train['deg_C'])/(243.12+df_train['deg_C']))/(17.62-(np.log(df_train['relative_humidity'] * 0.01)+17.62*df_train['deg_C']/(243.12+df_train['deg_C'])))

In [ ]:
data1['date_time'] = pd.to_datetime(data1['date_time'])
data1['year'] = data1['date_time'].dt.year
data1['month'] = data1['date_time'].dt.month
data1['week'] = data1['date_time'].dt.week
data1['day'] = data1['date_time'].dt.day
data1['dayofweek'] = data1['date_time'].dt.dayofweek
data1['time'] = data1['date_time'].dt.date - data1['date_time'].dt.date.min()
data1['hour'] = data1['date_time'].dt.hour
data1['time'] = data1['time'].apply(lambda x : x.days)
data1.drop(columns = 'date_time', inplace = True)
data1['SMC'] = (data1['absolute_humidity'] * 100) / data1['relative_humidity']
data1['Dew_Point'] = 243.12*(np.log(data1['relative_humidity'] * 0.01) + (17.62 * data1['deg_C'])/(243.12+data1['deg_C']))/(17.62-(np.log(data1['relative_humidity'] * 0.01)+17.62*data1['deg_C']/(243.12+data1['deg_C'])))

data2['date_time'] = pd.to_datetime(data2['date_time'])
data2['year'] = data2['date_time'].dt.year
data2['month'] = data2['date_time'].dt.month
data2['week'] = data2['date_time'].dt.week
data2['day'] = data2['date_time'].dt.day
data2['dayofweek'] = data2['date_time'].dt.dayofweek
data2['time'] = data2['date_time'].dt.date - data2['date_time'].dt.date.min()
data2['hour'] = data2['date_time'].dt.hour
data2['time'] = data2['time'].apply(lambda x : x.days)
data2.drop(columns = 'date_time', inplace = True)
data2['SMC'] = (data2['absolute_humidity'] * 100) / data2['relative_humidity']
data2['Dew_Point'] = 243.12*(np.log(data2['relative_humidity'] * 0.01) + (17.62 * data2['deg_C'])/(243.12+data2['deg_C']))/(17.62-(np.log(data2['relative_humidity'] * 0.01)+17.62*data2['deg_C']/(243.12+data2['deg_C'])))

data3['date_time'] = pd.to_datetime(data3['date_time'])
data3['year'] = data3['date_time'].dt.year
data3['month'] = data3['date_time'].dt.month
data3['week'] = data3['date_time'].dt.week
data3['day'] = data3['date_time'].dt.day
data3['dayofweek'] = data3['date_time'].dt.dayofweek
data3['time'] = data3['date_time'].dt.date - data3['date_time'].dt.date.min()
data3['hour'] = data3['date_time'].dt.hour
data3['time'] = data3['time'].apply(lambda x : x.days)
data3.drop(columns = 'date_time', inplace = True)
data3['SMC'] = (data3['absolute_humidity'] * 100) / data3['relative_humidity']
data3['Dew_Point'] = 243.12*(np.log(data3['relative_humidity'] * 0.01) + (17.62 * data3['deg_C'])/(243.12+data3['deg_C']))/(17.62-(np.log(data3['relative_humidity'] * 0.01)+17.62*data3['deg_C']/(243.12+data3['deg_C'])))

In [ ]:

fig, ax = plt.subplots(5, 3, figsize = (10,13))

ax[0,0].plot(df_train.groupby(df_train['sensor_1'])['target_carbon_monoxide'].mean(), 'r');
ax[0,1].plot(df_train.groupby(df_train['sensor_1'])['target_benzene'].mean(), 'r');
ax[0,2].plot(df_train.groupby(df_train['sensor_1'])['target_nitrogen_oxides'].mean(), 'r');

ax[1,0].plot(df_train.groupby(df_train['sensor_2'])['target_carbon_monoxide'].mean(), 'b');
ax[1,1].plot(df_train.groupby(df_train['sensor_2'])['target_benzene'].mean(), 'b');
ax[1,2].plot(df_train.groupby(df_train['sensor_2'])['target_nitrogen_oxides'].mean(), 'b');

ax[2,0].plot(df_train.groupby(df_train['sensor_3'])['target_carbon_monoxide'].mean(), 'y');
ax[2,1].plot(df_train.groupby(df_train['sensor_3'])['target_benzene'].mean(), 'y');
ax[2,2].plot(df_train.groupby(df_train['sensor_3'])['target_nitrogen_oxides'].mean(), 'y');

ax[3,0].plot(df_train.groupby(df_train['sensor_4'])['target_carbon_monoxide'].mean(), 'black');
ax[3,1].plot(df_train.groupby(df_train['sensor_4'])['target_benzene'].mean(), 'black');
ax[3,2].plot(df_train.groupby(df_train['sensor_4'])['target_nitrogen_oxides'].mean(), 'black');

ax[4,0].plot(df_train.groupby(df_train['sensor_5'])['target_carbon_monoxide'].mean(), 'violet');
ax[4,1].plot(df_train.groupby(df_train['sensor_5'])['target_benzene'].mean(), 'violet');
ax[4,2].plot(df_train.groupby(df_train['sensor_5'])['target_nitrogen_oxides'].mean(), 'violet');

ax[0,0].set_title('sensor_1-CO')
ax[0,1].set_title('sensor_1-Benzene')
ax[0,2].set_title('sensor_1-NOx')

ax[1,0].set_title('sensor_2-CO')
ax[1,1].set_title('sensor_2-Benzene')
ax[1,2].set_title('sensor_2-NOx')

ax[2,0].set_title('sensor_3-CO')
ax[2,1].set_title('sensor_3-Benzene')
ax[2,2].set_title('sensor_3-NOx')

ax[3,0].set_title('sensor_4-CO')
ax[3,1].set_title('sensor_4-Benzene')
ax[3,2].set_title('sensor_4-NOx')

ax[4,0].set_title('sensor_5-CO')
ax[4,1].set_title('sensor_5-Benzene')
ax[4,2].set_title('sensor_5-NOx')

fig.tight_layout()
plt.show()

df_train_cop = df_train[df_train["sensor_2"]>420]

df_train_2 = df_train.copy();

df_train_2["is_faulty"] = (df_train["sensor_2"]<420).astype(int)


In [ ]:
target = "is_faulty"

features = data1.columns.drop(target)



In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data1[features],data1[target],test_size=0.2,random_state=69420)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import time

FOLDS = 5
xgb_params = {
    'objective' : "binary:logistic",
    'eval_metric' : "error",
    'tree_method' : 'gpu_hist',
    'predictor' : 'gpu_predictor',
    'scale_pos_weight' : 15,
    'max_depth' : 3
}

xgb_predictions = []
xgb_scores = []
xgb_featureimp = []
models = []

skf = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state=69420)
for fold,(train_idx,val_idx) in enumerate(skf.split(x_train,y_train)):
    print(10*"-",f"FOLD = {fold+1}",10*"-")
    start_time = time.time()
    
    X_train, X_val = x_train.iloc[train_idx][features],x_train.iloc[val_idx][features]
    Y_train, Y_val = y_train.iloc[train_idx],y_train.iloc[val_idx]
    
    model = XGBClassifier(**xgb_params)
    model.fit(X_train, Y_train, verbose=0)
    
    preds_val = model.predict(X_val)
    f1 = f1_score(Y_val,preds_val)
    xgb_scores.append(f1)
    preds_test = model.predict(x_test)
    xgb_predictions.append(preds_test)
    
    run_time = time.time()-start_time
    
    models.append(model)
    
    print(f"Fold: {fold+1}, F1: {f1:.5f}, Run Time: {run_time:.2f}s")


In [ ]:
model = models[0]

import matplotlib.pyplot as plt


plt.barh(features,model.feature_importances_)



In [ ]:
from sklearn.metrics import f1_score
from scipy.stats import mode

preds_test = np.squeeze(mode(np.column_stack(xgb_predictions),axis = 1)[0]).astype('int')
f1_score(y_test,preds_test)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data2[features],data2[target],test_size=0.2,random_state=69420)

In [ ]:
FOLDS = 5
xgb_params = {
    'objective' : "binary:logistic",
    'eval_metric' : "error",
    'tree_method' : 'gpu_hist',
    'predictor' : 'gpu_predictor',
    'scale_pos_weight' : 15,
    'max_depth' : 3
}

xgb_predictions = []
xgb_scores = []
xgb_featureimp = []
models = []

skf = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state=69420)
for fold,(train_idx,val_idx) in enumerate(skf.split(x_train,y_train)):
    print(10*"-",f"FOLD = {fold+1}",10*"-")
    start_time = time.time()
    
    X_train, X_val = x_train.iloc[train_idx][features],x_train.iloc[val_idx][features]
    Y_train, Y_val = y_train.iloc[train_idx],y_train.iloc[val_idx]
    
    model = XGBClassifier(**xgb_params)
    model.fit(X_train, Y_train, verbose=0)
    
    preds_val = model.predict(X_val)
    f1 = f1_score(Y_val,preds_val)
    xgb_scores.append(f1)
    preds_test = model.predict(x_test)
    xgb_predictions.append(preds_test)
    
    run_time = time.time()-start_time
    
    models.append(model)
    
    print(f"Fold: {fold+1}, F1: {f1:.5f}, Run Time: {run_time:.2f}s")



In [ ]:
preds_test = np.squeeze(mode(np.column_stack(xgb_predictions),axis = 1)[0]).astype('int')
f1_score(y_test,preds_test)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data3[features],data2[target],test_size=0.2,random_state=69420)

In [ ]:
FOLDS = 5
xgb_params = {
    'objective' : "binary:logistic",
    'eval_metric' : "error",
    'tree_method' : 'gpu_hist',
    'predictor' : 'gpu_predictor',
    'scale_pos_weight' : 15,
    'max_depth' : 3
}

xgb_predictions = []
xgb_scores = []
xgb_featureimp = []
models = []

skf = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state=69420)
for fold,(train_idx,val_idx) in enumerate(skf.split(x_train,y_train)):
    print(10*"-",f"FOLD = {fold+1}",10*"-")
    start_time = time.time()
    
    X_train, X_val = x_train.iloc[train_idx][features],x_train.iloc[val_idx][features]
    Y_train, Y_val = y_train.iloc[train_idx],y_train.iloc[val_idx]
    
    model = XGBClassifier(**xgb_params)
    model.fit(X_train, Y_train, verbose=0)
    
    preds_val = model.predict(X_val)
    f1 = f1_score(Y_val,preds_val)
    xgb_scores.append(f1)
    preds_test = model.predict(x_test)
    xgb_predictions.append(preds_test)
    
    run_time = time.time()-start_time
    
    models.append(model)
    
    print(f"Fold: {fold+1}, F1: {f1:.5f}, Run Time: {run_time:.2f}s")


In [ ]:
preds_test = np.squeeze(mode(np.column_stack(xgb_predictions),axis = 1)[0]).astype('int')
f1_score(y_test,preds_test)